# User Guide

## Overview

Importing `xarray_bounds` registers a {py:class}`~xarray.Dataset` accessor named `bnds`. The `bnds` accessor provides utilities focused around 3 key operations:

1. [Access](#access) for bounds lookup;
2. [Assign](#assign) for creating and/or assigning bounds;
3. [Drop](#drop) for removing bounds.

First, let’s load a dataset that contains boundary coordinates.

The `simple_bounds` dataset is a synthetic CF-compliant dataset with independent latitude, longitude and time axes.

In [1]:
import xarray as xr
import xarray_bounds as xrb

xr.set_options(keep_attrs=True)

ds = xrb.datasets.simple_bounds
ds

<xarray.Dataset> Size: 424B
Dimensions:      (time: 4, lat: 3, lon: 2, bnds: 2)
Coordinates:
  * time         (time) datetime64[ns] 32B 2000-01-01T12:00:00 ... 2000-01-04...
  * lat          (lat) int64 24B 90 100 110
  * lon          (lon) int64 16B 180 190
    aux          (time) <U1 16B 'a' 'b' 'c' 'd'
    time_bnds    (time, bnds) datetime64[ns] 64B 2000-01-01 ... 2000-01-05
    lat_bnds     (lat, bnds) int64 48B 85 95 95 105 105 115
    lon_bnds     (lon, bnds) int64 32B 175 185 185 195
Dimensions without coordinates: bnds
Data variables:
    temperature  (time, lat, lon) float64 192B 208.6 254.3 131.7 ... 93.48 238.1
Attributes:
    description:  A synthetic CF-compliant dataset with independent coordinat...
    references:   https://cfconventions.org/cf-conventions/cf-conventions.htm...
    see_also:     xarray_bounds.datasets.simple_bounds

(access)=
## Accessing bounds

The `bnds` accessor returns a {py:class}`~collections.abc.Mapping` of variable names to boundary coordinates.

In [2]:
ds.bnds

Bounds:
    lon      (lon, bnds) int64 32B 175 185 185 195
    time     (time, bnds) datetime64[ns] 64B 2000-01-01 ... 2000-01-05
    lat      (lat, bnds) int64 48B 85 95 95 105 105 115

The `bnds` accessor is loosely modelled on the {py:class}`xarray.Coordinates` object. One important difference is that the keys are the names of the coordinates the bounds describe and **not** the bounds themselves.

:::{note}
Boundary variables are accessed by the names of the variables they describe and **not** by their own variable names.
:::

In [3]:
ds.bnds['time']

<xarray.DataArray 'time_bnds' (time: 4, bnds: 2)> Size: 64B
array([['2000-01-01T00:00:00.000000000', '2000-01-02T00:00:00.000000000'],
       ['2000-01-02T00:00:00.000000000', '2000-01-03T00:00:00.000000000'],
       ['2000-01-03T00:00:00.000000000', '2000-01-04T00:00:00.000000000'],
       ['2000-01-04T00:00:00.000000000', '2000-01-05T00:00:00.000000000']],
      dtype='datetime64[ns]')
Coordinates:
  * time       (time) datetime64[ns] 32B 2000-01-01T12:00:00 ... 2000-01-04T1...
    aux        (time) <U1 16B 'a' 'b' 'c' 'd'
    time_bnds  (time, bnds) datetime64[ns] 64B 2000-01-01 ... 2000-01-05
Dimensions without coordinates: bnds

If you want to access the bounds by their variable names, you can just use {py:attr}`~xarray.Dataset.coords` like normal.

```python
ds.coords['time_bnds']
```

## Aliases 

The ``bnds`` mapping supports using key aliases. You can use any name understood by [cf_xarray](https://cf-xarray.readthedocs.io/en/latest/quickstart.html#using-attributes) to refer to variables. Aliases are supported for item access, membership checks and as arguments for all of `bnds`'s methods.

Aliase types:
- variable name (usual xarray names);
- CF standard name (e.g. longitude, latitude, time);
- CF axis name (one of {X, Y, Z, T}). 


In [4]:
# Variable name
name = 'lat'

aliases = [
    # CF Standard name
    'latitude',
    # CF Axis name
    'Y',
]

for alias in aliases:
    assert alias in ds.bnds

for alias in aliases:
    # square bracket notation
    xr.testing.assert_identical(
        ds.bnds[alias],
        ds.bnds[name],
    )
    # .get() method
    xr.testing.assert_identical(
        ds.bnds.get(alias),
        ds.bnds.get(name),
    )


Aliases are ignored during iteration. Iterating over the mapping only traverses the canonical keys.

In [5]:
for name in ds.bnds:
    print(name)
print(len(ds.bnds))

lon
time
lat
3


(drop)=
## Dropping bounds

To drop an object's bounds, we can use the {py:meth}`~xarray.Dataset.bnds.drop_bounds` method.

In [6]:
ds2 = ds.copy()
ds2 = ds2.bnds.drop_bounds('Y')
ds2.bnds

Bounds:
    lon      (lon, bnds) int64 32B 175 185 185 195
    time     (time, bnds) datetime64[ns] 64B 2000-01-01 ... 2000-01-05

Passing no arguments to {py:meth}`~xarray.Dataset.bnds.drop_bounds` will drop all available bounds.

In [7]:
ds2 = ds2.bnds.drop_bounds()
ds2.bnds

Bounds:
    *empty*

In addition to dropping the specified bounds, the method also removes appropriate CF metadata attributes.

In [8]:
'bounds' in ds2.time.attrs

False

(assign)=
## Adding bounds

### Inferring bounds

For dimension coordinates with regular* indexes, we can use {py:meth}`~xarray.Dataset.bnds.infer_bounds()` to automagically add boundary variables.

In [9]:
ds2 = ds2.bnds.infer_bounds('time')
ds2.bnds['time']

<xarray.DataArray 'time_bnds' (time: 4, bnds: 2)> Size: 64B
array([['2000-01-01T12:00:00.000000000', '2000-01-02T12:00:00.000000000'],
       ['2000-01-02T12:00:00.000000000', '2000-01-03T12:00:00.000000000'],
       ['2000-01-03T12:00:00.000000000', '2000-01-04T12:00:00.000000000'],
       ['2000-01-04T12:00:00.000000000', '2000-01-05T12:00:00.000000000']],
      dtype='datetime64[ns]')
Coordinates:
  * time       (time) datetime64[ns] 32B 2000-01-01T12:00:00 ... 2000-01-04T1...
    aux        (time) <U1 16B 'a' 'b' 'c' 'd'
    time_bnds  (time, bnds) datetime64[ns] 64B 2000-01-01T12:00:00 ... 2000-0...
Dimensions without coordinates: bnds
Attributes:
    closed:   left

{py:meth}`~xarray.Dataset.bnds.infer_bounds()` can infer bounds for {py:class}`pandas.DatetimeIndex` and numeric {py:class}`pandas.Index` indexes. For datetime indexes, the index *must* have an inferrable frequency, while numeric indexes are *assumed* to be equally spaced.

To control how bounds are inferred, we can specify which side of the bounds the index labels and which side of the bounds interval is closed.

In [10]:
ds2 = ds2.bnds.infer_bounds('Y', label='middle', closed='left')
ds2.bnds['lat']

<xarray.DataArray 'lat_bnds' (lat: 3, bnds: 2)> Size: 48B
array([[ 85.,  95.],
       [ 95., 105.],
       [105., 115.]])
Coordinates:
  * lat       (lat) int64 24B 90 100 110
    lat_bnds  (lat, bnds) float64 48B 85.0 95.0 95.0 105.0 105.0 115.0
Dimensions without coordinates: bnds
Attributes:
    closed:   left

The function tries to derive sensible defaults if the `label` and/or `closed` arguments are not provided.

### Custom bounds

To add existing bounds objects, we can use {py:meth}`~xarray.Dataset.bnds.assign_bounds()`. 

In [12]:
lon_bnds = xr.DataArray(
    data=[[175, 185], [185, 195]],
    dims=('lon', 'bnds'),
)
ds2 = ds2.bnds.assign_bounds(X=lon_bnds)
ds2.bnds['lon']

<xarray.DataArray 'lon_bnds' (lon: 2, bnds: 2)> Size: 32B
array([[175, 185],
       [185, 195]])
Coordinates:
  * lon       (lon) int64 16B 180 190
    lon_bnds  (lon, bnds) int64 32B 175 185 185 195
Dimensions without coordinates: bnds

This method will assign the bounds and add appropriate CF metadata attributes.

In [15]:
for c in ['time', 'lat', 'lon']:
    print(c, ds2[c].attrs, sep=': ')

time: {'standard_name': 'time', 'axis': 'T', 'bounds': 'time_bnds'}
lat: {'standard_name': 'latitude', 'axis': 'Y', 'units': 'degrees_north', 'bounds': 'lat_bnds'}
lon: {'standard_name': 'longitude', 'axis': 'X', 'units': 'degrees_east', 'bounds': 'lon_bnds'}


## Configuration

`xarray_bounds` offers a small number of configuration options through {py:func}`~xarray_bounds.set_options`. Currently, you can:

1. Control the name of the vertex dimension: `bounds_dim`

You can set these options either globally:

In [ ]:
xrb.set_options(bounds_dim='bounds')

or locally using a context manager:

In [ ]:
with xrb.set_options(bounds_dim='bounds'):
    # ... do bounds operation here ...
    pass

## Core functions

Most of the core operations exposed by the accessor are also available as [utility functions](api). These functions are particularly useful if you don't want to use the accessor interface, or if you want to create bounds for datasets that do not conform to the CF convention.